In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dane/_info-data-discrete.txt', sep=' ', header=None, names=["nazwa systemu", "liczba atrybutow", "liczba obiektow"])
df

,nazwa systemu,liczba atrybutow,liczba obiektow
0,australian,15,690
1,car,7,1728
2,fertilityDiagnosis,10,100
3,german-credit,21,1000
4,heartdisease,14,270
5,hepatitis-filled,20,155
6,house-votes-84,17,435
7,mushroom-modified-filled,23,8124
8,nursery,9,12960
9,diabetes,9,768


In [3]:
attributes_type = pd.read_csv("dane/australian-type.txt", sep=" ", header=None, names=["Attribute", "Type"])
attributes_type

,Attribute,Type
0,a1,s
1,a2,n
2,a3,n
3,a4,s
4,a5,s
5,a6,s
6,a7,n
7,a8,s
8,a9,s
9,a10,n


In [4]:
classes = attributes_type['Attribute']
classes = classes.squeeze() 
new_row = pd.Series(["Decyzja"], index=[len(classes)])
classes = pd.concat([classes, new_row])
systems_info = pd.read_csv('dane/australian.txt', sep=' ', header=None)
systems_info.columns = classes.values
systems_info

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,Decyzja
0,1.234,22.08,11.46,2,4,4,1.585,0,0,0,1,2,100,1213,0
1,0.000,22.67,3.80,2,8,4,0.165,0,0,0,0,2,160,1,0
2,0.000,29.58,1.75,1,4,4,1.250,0,0,0,1,2,280,1,0
3,0.000,21.67,11.50,1,5,3,0.000,1,1,11,1,2,0,1,1
4,1.000,20.17,8.17,2,6,4,1.960,1,1,14,0,2,60,159,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1.000,31.57,10.50,2,14,4,6.500,1,0,0,0,2,0,1,1
686,1.000,20.67,0.41,2,8,4,0.125,0,0,0,0,2,0,45,0
687,0.000,18.82,9.53,2,6,4,0.085,1,0,0,0,2,100,1,1
688,0.000,27.42,14.50,2,14,8,3.085,1,1,1,0,2,120,12,1


# 3) Wczytujemy wybrany system np. w C++ i zdobywamy następujące informacje o systemie (load selected decision system for instance in C++ and find the information):

## a) wypisujemy istniejące w systemie symbole klas decyzyjnych (find available decision classes),

In [5]:
decision_classes = systems_info.iloc[:, -1].unique()
print("Symbole klas decyzyjnych:", decision_classes)

Symbole klas decyzyjnych: [0 1]


## b) wielkości klas decyzyjnych (liczby obiektów w klasach) (find size of decision classes (number of objects in classes) ),

In [6]:
class_counts = systems_info.iloc[:, -1].value_counts()
print("Liczba obiektów w klasach:\n", class_counts)

Liczba obiektów w klasach:
 0    383
1    307
Name: Decyzja, dtype: int64


## c) minimalne i maksymalne wartości poszczególnych atrybutów(dotyczy atrybutów numerycznych) (minimal and maximal values for each attribute - apply for numerical attributes),

In [7]:
min_max_attributes = pd.DataFrame()

for _, row in attributes_type.iterrows():
    if row['Type'] == "n":
        min_max_attributes[row['Attribute']] = [systems_info[row['Attribute']].min(), systems_info[row['Attribute']].max()]

min_max_attributes.index = ["Min", "Max"]

display(min_max_attributes)
        

,a2,a3,a7,a10,a13,a14
Min,13.75,0.0,0.0,0,0,1
Max,80.25,28.0,28.5,67,2000,100001


## d) dla każdego atrybutu wypisujemy liczbę różnych dostępnych wartości (for each attribute detect the number of different available values),

In [8]:
for column in systems_info.columns[:-1]:
    print(f"Columna '{column}' zawiera {systems_info[column].nunique()} unikalne wartości")


Columna 'a1' zawiera 3 unikalne wartości
Columna 'a2' zawiera 350 unikalne wartości
Columna 'a3' zawiera 212 unikalne wartości
Columna 'a4' zawiera 3 unikalne wartości
Columna 'a5' zawiera 14 unikalne wartości
Columna 'a6' zawiera 8 unikalne wartości
Columna 'a7' zawiera 132 unikalne wartości
Columna 'a8' zawiera 2 unikalne wartości
Columna 'a9' zawiera 2 unikalne wartości
Columna 'a10' zawiera 23 unikalne wartości
Columna 'a11' zawiera 2 unikalne wartości
Columna 'a12' zawiera 3 unikalne wartości
Columna 'a13' zawiera 171 unikalne wartości
Columna 'a14' zawiera 240 unikalne wartości


## e) dla każdego atrybutu wypisujemy listę wszystkich różnych dostępnych wartości(for each attribute list the set of different, available values),

In [9]:
for column in systems_info.columns[:-1]:
    print(f"Atrybut '{column}': {systems_info[column].unique()}")

Atrybut 'a1': [1.234 0.    1.   ]
Atrybut 'a2': [22.08 22.67 29.58 21.67 20.17 15.83 17.42 58.67 27.83 55.75 33.5  41.42
 20.67 34.92 58.58 48.08 18.92 20.   22.42 28.17 19.17 41.17 41.58 19.5
 32.75 22.5  33.17 30.67 23.08 27.   20.42 52.33 42.83 74.83 25.   39.58
 47.75 47.42 23.17 22.58 26.75 63.33 23.75 20.75 24.5  16.17 29.5  52.83
 32.33 21.08 19.   27.58 40.   37.33 42.5  56.75 43.17 18.5  40.83 42.
 21.5  31.25 27.25 48.75 30.42 29.42 28.25 40.25 36.5  25.58 29.83 32.17
 25.17 35.17 18.57 39.92 23.42 37.58 24.75 47.   34.17 22.17 27.75 42.75
 28.67 36.25 18.17 21.25 38.92 31.83 17.32 39.08 38.67 27.67 22.25 49.83
 38.25 47.33 35.75 33.58 34.08 22.75 40.58 38.75 57.08 22.   58.33 28.92
 46.   21.   20.83 24.58 26.5  40.92 38.33 19.57 39.25 25.75 46.08 19.67
 18.82 64.08 16.5  68.67 76.75 15.92 34.83 45.17 15.17 52.5  18.   37.5
 47.83 33.08 43.08 34.5  18.25 17.92 18.42 62.5  23.5  35.25 56.83 53.33
 42.17 33.75 25.67 24.33 23.33 37.17 26.25 29.75 23.   17.25 29.25 28.58
 34.58 

## f)odchylenie standardowe dla poszczególnych atrybutów w całym systemie i w klasach decyzyjnych (dotyczy atrybutów numerycznych) (compute standard deviation for each attribute in the whole system and separately for each decision class).

In [10]:
import statistics

print("Odchylenie standardowe całego systemu: ", systems_info.std())

min_max_attributes = pd.DataFrame()

for _, row in attributes_type.iterrows():
    if row['Type'] == "n":
        min_max_attributes[row['Attribute']] = [statistics.stdev(systems_info[row['Attribute']])]

min_max_attributes.index = ["Odchylenie standardowe"]

display(min_max_attributes)

Odchylenie standardowe całego systemu:  a1            0.467801
a2           11.853826
a3            4.977828
a4            0.430063
a5            3.683265
a6            1.992316
a7            3.346513
a8            0.499824
a9            0.495080
a10           4.862940
a11           0.498592
a12           0.298813
a13         172.159274
a14        5210.102598
Decyzja       0.497318
dtype: float64


,a2,a3,a7,a10,a13,a14
Odchylenie standardowe,11.853826,4.977828,3.346513,4.86294,172.159274,5210.102598


# 4) Wykonaj dla wybranych danych następujący preprocessing (do for selected data the following preprocessing):

## 4a) Generowanie 10% wartości nieznanych i ich uzupełnienie

In [11]:
num_rows = len(systems_info)
num_new_rows = int(0.1 * num_rows) 

# Tworzenie nowych wierszy z brakującymi wartościami
new_rows = pd.DataFrame({col: '?' for col in systems_info.columns}, index=range(num_rows, num_rows + num_new_rows))

for column in new_rows.columns[:-1]: 
    if systems_info[column].dtype == 'int64' or systems_info[column].dtype == 'float64':
        new_rows[column] = systems_info[column].mean() 
    else:
        new_rows[column] = systems_info[column].mode()[0]
            
new_rows

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,Decyzja
690,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
691,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
692,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
693,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
694,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
755,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
756,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
757,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?


In [12]:
systems_info = pd.concat([systems_info, new_rows], ignore_index=True)
systems_info

,a1,a2,a3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,Decyzja
0,1.2340,22.08000,11.460000,2.000000,4.000000,4.000000,1.585000,0.000000,0.000000,0.0,1.000000,2.000000,100.000000,1213.000000,0
1,0.0000,22.67000,3.800000,2.000000,8.000000,4.000000,0.165000,0.000000,0.000000,0.0,0.000000,2.000000,160.000000,1.000000,0
2,0.0000,29.58000,1.750000,1.000000,4.000000,4.000000,1.250000,0.000000,0.000000,0.0,1.000000,2.000000,280.000000,1.000000,0
3,0.0000,21.67000,11.500000,1.000000,5.000000,3.000000,0.000000,1.000000,1.000000,11.0,1.000000,2.000000,0.000000,1.000000,1
4,1.0000,20.17000,8.170000,2.000000,6.000000,4.000000,1.960000,1.000000,1.000000,14.0,0.000000,2.000000,60.000000,159.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
755,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
756,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?
757,0.6786,31.56771,4.752565,1.766667,7.372464,4.692754,2.223406,0.523188,0.427536,2.4,0.457971,1.928986,184.014493,1018.385507,?


## 4b) Normalizacja atrybutów numerycznych

In [13]:
normalized_dfs = []
for column in systems_info.columns[:-1]:
        min_val, max_val = systems_info[column].min(), systems_info[column].max()
        df_norm = pd.DataFrame({
            f'{column}_original': systems_info[column],
            f'{column}_norm_-1_1': 2 * (systems_info[column] - min_val) / (max_val - min_val) - 1,
            f'{column}_norm_0_1': (systems_info[column] - min_val) / (max_val - min_val),
            f'{column}_norm_-10_10': 20 * (systems_info[column] - min_val) / (max_val - min_val) - 10
        })
        normalized_dfs.append(df_norm)

final_normalized_data = pd.concat(normalized_dfs, axis=1)
final_normalized_data.head()

,a1_original,a1_norm_-1_1,a1_norm_0_1,a1_norm_-10_10,a2_original,a2_norm_-1_1,a2_norm_0_1,a2_norm_-10_10,a3_original,a3_norm_-1_1,...,a12_norm_0_1,a12_norm_-10_10,a13_original,a13_norm_-1_1,a13_norm_0_1,a13_norm_-10_10,a14_original,a14_norm_-1_1,a14_norm_0_1,a14_norm_-10_10
0,1.234,1.000000,1.000000,10.000000,22.08,-0.749474,0.125263,-7.494737,11.46,-0.181429,...,0.5,0.0,100.0,-0.90,0.05,-9.0,1213.0,-0.97576,0.01212,-9.7576
1,0.000,-1.000000,0.000000,-10.000000,22.67,-0.731729,0.134135,-7.317293,3.80,-0.728571,...,0.5,0.0,160.0,-0.84,0.08,-8.4,1.0,-1.00000,0.00000,-10.0000
2,0.000,-1.000000,0.000000,-10.000000,29.58,-0.523910,0.238045,-5.239098,1.75,-0.875000,...,0.5,0.0,280.0,-0.72,0.14,-7.2,1.0,-1.00000,0.00000,-10.0000
3,0.000,-1.000000,0.000000,-10.000000,21.67,-0.761805,0.119098,-7.618045,11.50,-0.178571,...,0.5,0.0,0.0,-1.00,0.00,-10.0,1.0,-1.00000,0.00000,-10.0000
4,1.000,0.620746,0.810373,6.207455,20.17,-0.806917,0.096541,-8.069173,8.17,-0.416429,...,0.5,0.0,60.0,-0.94,0.03,-9.4,159.0,-0.99684,0.00158,-9.9684


## 4c) Standaryzacja wartości numerycznych

In [14]:
standardized_dfs = []
for column in systems_info.columns[:-1]:
    mean, std = systems_info[column].mean(), systems_info[column].std()
    df_std = pd.DataFrame({
        f'{column}_original': systems_info[column],
        f'{column}_standardized': (systems_info[column] - mean) / std
    })
    standardized_dfs.append(df_std)

standardized_dfs

[     a1_original  a1_standardized
 0         1.2340         1.245288
 1         0.0000        -1.521520
 2         0.0000        -1.521520
 3         0.0000        -1.521520
 4         1.0000         0.720626
 ..           ...              ...
 754       0.6786         0.000000
 755       0.6786         0.000000
 756       0.6786         0.000000
 757       0.6786         0.000000
 758       0.6786         0.000000
 
 [759 rows x 2 columns],
      a2_original  a2_standardized
 0       22.08000    -8.395139e-01
 1       22.67000    -7.873081e-01
 2       29.58000    -1.758812e-01
 3       21.67000    -8.757924e-01
 4       20.17000    -1.008519e+00
 ..           ...              ...
 754     31.56771     3.143596e-16
 755     31.56771     3.143596e-16
 756     31.56771     3.143596e-16
 757     31.56771     3.143596e-16
 758     31.56771     3.143596e-16
 
 [759 rows x 2 columns],
      a3_original  a3_standardized
 0      11.460000     1.413324e+00
 1       3.800000    -2.007150e-01
 

## 4d) Przeformatowanie danych i zamiana atrybutu symbolicznego na Dummy Variables